#### Siyuan "Kevin" Peng, Yuanzhe "Siris" Zheng, Yanlin "Jacky" Liu
![Image of Yaktocat](https://cdn.mos.cms.futurecdn.net/rLh7Dh7EKo8F6zmDtXYp8W.jpg)
# Table of Contents
1. [Introduction](#introduction)
2. [Install packages](#install-pkg)
3. [Data Dowloading](#data-download)
4. [Preprocessing](#preprocessing)<br>
    a. [Load and Clean Dataset](#load-and-clean)<br>
    b. [Data Analysis and Visualization](#data-ana-vis)
5. [Machine Learning Model](#ml-model)<br>
    a. [What and Why](#what-why)<br>
    b. [Training](#training)<br>
    c. [Result Anlysis and Demonstration](#result-ana-demon)
6. [Future Application](#future-app)
7. [Reference and External Link](#ref-and-extlink)

## 1. Introduction <a name="introduction"></a>
TODO:

## 2. Install Packages <a name="install-pkg"></a>
```
pip install kaggle numpy matplotlib pandas sklearn
```
or use environment.yml to install packages in conda environment
```
conda env update -f environment.yml
```
## 3. Data Downloading <a name="data-download"></a>

In [3]:
import kaggle
# remember to put kaggle.json to your C:/username/.kaggle
!kaggle datasets download -d ashaheedq/video-games-sales-2019

video-games-sales-2019.zip: Skipping, found more recently modified local copy (use --force to force download)


or directly download from kaggle webpage: [https://www.kaggle.com/ashaheedq/video-games-sales-2019](https://www.kaggle.com/ashaheedq/video-games-sales-2019)_
## 4. Preprocessing <a name="preprocessing"></a>
### 4.a Load and Clean Data <a name="load-and-clean"></a>

In [8]:
# import necessary packages
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
import dataframe_image as dfi
df = pd.read_csv("vgsales-12-4-2019.csv")
df = df.drop(columns=['basename', 'Total_Shipped', 'Platform', 'Publisher', 'VGChartz_Score', 'Last_Update', 'url', 'status', 'Vgchartzscore', 'img_url', 'ESRB_Rating'])
df = df.dropna(subset=['NA_Sales', 'Developer', 'Genre'])
df = df.reset_index()
for i in range(len(df)):
    df.loc[i, 'Developer'] = str(df.loc[i, 'Developer'])

le = LabelEncoder()
ohe = OneHotEncoder(handle_unknown = 'ignore')
df['Developer'] = le.fit_transform(df['Developer'])
df['Genre'] = le.fit_transform(df['Genre'])
# df_genre = pd.DataFrame(ohe.fit_transform(df[['Genre']]).toarray())
# df_developer = pd.DataFrame(ohe.fit_transform(df[['Developer']]).toarray())
df_temp = pd.DataFrame(ohe.fit_transform(df[['Genre', 'Developer']]).toarray())

In [10]:
df = df.join(df_temp)
df.shape
# dfi.export(df.head(), 'images/df_styled.png')

ValueError: columns overlap but no suffix specified: Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       2211, 2212, 2213, 2214, 2215, 2216, 2217, 2218, 2219, 2220],
      dtype='object', length=2221)

![](images/df_styled.png)

### 4.b Data Analysis and Visualization' <a name="data-ana-vis"></a>

In [5]:
#TODO:

## 5. Machine Learning Model <a name="ml-model"></a>
### 5.a What and Why <a name="what-why"></a>

In [6]:
#TODO:

### 5.b Training <a name="training"></a>

In [7]:
#TODO:

### 5.c Result Anlysis and Demonstration <a name="result-ana-demon"></a>
TODO:
## 6. Future Application <a name="future-app"></a>
TODO:
## 7. Reference and External Link <a name="ref-and-extlink"></a>
TODO: